In [1]:
# %env CUDA_VISIBLE_DEVICES=0
import numpy as np
import jax
import jax.numpy as jnp
from jax import jit
from functools import partial

from ckks.encoder import CKKSEncoder
from ckks.cpu.enc_cpu import CKKSEncoder as CKKSEncoderCPU

ImportError: cannot import name 'CKKSEncoder' from 'ckks.encoder' (/home/jiajie/Repos/HE/ckks_jax/notebooks/ckks/encoder.py)

In [3]:
scale = 64
times = 4
M = 8*times

encoder = CKKSEncoder(M, scale)
z = jnp.array([3 + 4j, 2 - 1j]*times)
p = encoder.encode(z)
z_ = encoder.decode(p)
z_

DeviceArray([2.9947152+4.011139j , 1.9907174-1.0010103j,
             3.0099041+4.0146465j, 2.0128212-0.9828348j,
             2.9871714+4.0171556j, 1.990093 -0.9853475j,
             3.009272 +3.9989834j, 2.005281 -0.988852j ], dtype=complex64)

In [ ]:
%timeit jax.block_until_ready(encoder.encode(z))

11.8 ms ± 5.84 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
%timeit jax.block_until_ready(encoder.decode(p))

2 ms ± 27.3 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


## CPU

In [33]:
encoder_cpu = CKKSEncoderCPU(M, scale)
z_cpu = np.array([3 + 4j, 2 - 1j]*times)
p_cpu = encoder_cpu.encode(z_cpu)

In [34]:
%timeit encoder_cpu.encode(z_cpu)

87.6 ms ± 935 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [48]:
N = 50
key = jax.random.PRNGKey(0)
data = jax.random.uniform(key, (N, times*2))

@jit
def encode_n_gpu():
    data = jax.random.uniform(key, (N, times*2))
    return [encoder.encode(data[i,:]) for i in range(50)]

def encode_n_cpu():
    return [encoder_cpu.encode(z_cpu) for _ in range(50)]


In [50]:
%timeit jax.block_until_ready(encode_n_gpu())


113 ms ± 1.84 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [51]:
%timeit encode_n_cpu()

4.39 s ± 13.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
